In [ ]:
from gather_analysis_helper import *

In [ ]:
start_time = "2019-11-22 08:00:00"
end_time = "2019-11-22 12:00:00"

print("Start time: " + start_time)
print("End time: " + end_time)

In [ ]:
sn_rsn = {}
sn_first_seen = {}
sn_last_seen = {}
collect_start = 0
collect_end = 0
max_nodes = 80000
raw = make_file_iterator(start_time, end_time, os.path.join("data/raw.json"))
for status in raw:
    if "retweeted_status" in status:
        sn = status["user"]["screen_name"]
        rsn = status["retweeted_status"]["user"]["screen_name"]
        created_at = status["created_at"]
        tweet_time_unix = twitter_time_to_unix(created_at)
        if collect_start == 0:
            collect_start = tweet_time_unix
        if collect_end == 0:
            collect_end = tweet_time_unix
        if collect_start > tweet_time_unix:
            collect_start = tweet_time_unix
        if collect_end < tweet_time_unix:
            collect_end = tweet_time_unix
        if sn not in sn_rsn:
            sn_rsn[sn] = Counter()
        sn_rsn[sn][rsn] += 1
        if sn not in sn_first_seen:
            sn_first_seen[sn] = tweet_time_unix
        if sn not in sn_last_seen:
            sn_last_seen[sn] = tweet_time_unix
        if sn_first_seen[sn] > tweet_time_unix:
            sn_first_seen[sn] = tweet_time_unix
        if sn_last_seen[sn] < tweet_time_unix:
            sn_last_seen[sn] = tweet_time_unix
        if rsn not in sn_first_seen:
            sn_first_seen[rsn] = tweet_time_unix
        if rsn not in sn_last_seen:
            sn_last_seen[rsn] = tweet_time_unix
        if sn_first_seen[rsn] > tweet_time_unix:
            sn_first_seen[rsn] = tweet_time_unix
        if sn_last_seen[rsn] < tweet_time_unix:
            sn_last_seen[rsn] = tweet_time_unix
    if max_nodes > 0:
        if len(sn_rsn) > max_nodes:
            break
print(len(sn_rsn))

In [ ]:
print("Collection started on " + unix_time_to_readable(collect_start) + " and ended on " + unix_time_to_readable(collect_end))

In [ ]:
print("Getting communities")
clusters = get_communities(sn_rsn)
print("Making mapping")
mapping = []
node_names = set()
for source, targets in sn_rsn.items():
    node_names.add(source)
    for target, count in targets.items():
        node_names.add(target)
        mapping.append((source, target, count))
print("Done")

In [ ]:
print(len(sn_rsn))
start_times = {}
end_times = {}
for sn, start in sn_first_seen.items():
    end = sn_last_seen[sn]
    start_times[sn] = start
    if start == end:
        end = end + 180
    end_times[sn] = end

node_attr = {}
for name in node_names:
    node_attr[name] = []
attr_labels = []
    
# Community, as detected by Louvain method
for label, names in clusters.items():
    for name in names:
        node_attr[name].append(label)
        if name in sn_rsn:
            node_attr[name].append(len(sn_rsn[name]))
        else:
            node_attr[name].append(0)
attr_labels.append("community")
attr_labels.append("retweets")

write_timelapse_gexf(mapping, os.path.join("timelapse_graph.gexf"), start_times, end_times, node_attr, attr_labels)
print("Done")